# XMM-LSS master catalogue: Flags

In [1]:
import numpy as np
from astropy.table import Table, Column
import itertools

import glob

from herschelhelp_internal.flagging import flag_outliers

In [2]:
FIELD = 'XMM-LSS'
SUFFIX = '20180504'
# On Herschel-Stripe-82 we have a list of tiles due to the large size of the field. 
# Therefore, we here process each in turn
file_list = glob.glob('../../dmu1/dmu1_ml_XMM-LSS/data/tiles/sub_catalogue_xmm-lss_{}_*.fits'.format(SUFFIX))

In [3]:
g_bands = ["Megacam g", "DECam g", "SUPRIME g", "GPC1 g"]
r_bands = ["Megacam r", "DECam r", "SUPRIME r", "GPC1 r"]
i_bands = ["Megacam i", "DECam i", "SUPRIME i", "GPC1 i"]
z_bands = ["Megacam z", "DECam z", "SUPRIME z", "GPC1 z", "VISTA z"]
y_bands = ["Megacam y", "DECam y", "SUPRIME y", "GPC1 y", "VISTA y"]
J_bands = [                                               "VISTA J", "UKIDSS J", "WIRCAM J"]
H_bands = [                                               "VISTA H", "UKIDSS H", "WIRCAM H"]
K_bands = [                                               "VISTA Ks", "UKIDSS K", "WIRCAM Ks"]

all_bands = [g_bands, r_bands, i_bands, z_bands, y_bands, J_bands, H_bands, K_bands]

## 1. Magnitudes and magnitude erros

In [4]:
def flag_mag(master_catalogue, flagcol, mask):
    
    # Add flag columns if does not exist
    if flagcol not in master_catalogue.colnames:
        master_catalogue.add_column(Column(data=np.zeros(len(master_catalogue), dtype=bool), name=flagcol))
    
    # Flagged
    master_catalogue[flagcol][mask] = np.ones(len(mask), dtype=bool)
    print('    Number of flagged objects:', len(master_catalogue[flagcol][mask]))
    return master_catalogue

### 1.a Pan-STARRS Aperture and Total magnitude errors

In [5]:
def panstars_loopable(master_catalogue):
    ## dmu0: Pan-STARRS stack cat 
    gpc1_err = 0.0010860000038519502
    bands = ["GPC1 g", "GPC1 r", "GPC1 i", "GPC1 z", "GPC1 y"]

    for i, band in enumerate(bands):
        print(band)
        basecol = band.replace(" ", "_").lower()
    
        ecol_ap, ecol_tot = "merr_ap_{}".format(basecol), "merr_{}".format(basecol)
        flagcol_ap, flagcol_tot = "flag_{}_ap".format(basecol), "flag_{}".format(basecol)
    
        mask_ap  = np.where(master_catalogue[ecol_ap]  == gpc1_err)[0]
        mask_tot = np.where(master_catalogue[ecol_tot] == gpc1_err)[0]
    
        print('  Aperture magnitude')
        master_catalogue = flag_mag(master_catalogue, flagcol_ap, mask_ap)
        print('  Total magnitude')
        master_catalogue = flag_mag(master_catalogue, flagcol_tot, mask_tot)
        
    return master_catalogue
    

### 2.c IRAC Aperture magnitudes

In [6]:
def irac_loopable(master_catalogue):
    irac_mag = 3.9000000001085695
    bands = ["IRAC i1", "IRAC i2", "IRAC i3", "IRAC i4"]

    for i, band in enumerate(bands):
        print(band)
        basecol = band.replace(" ", "_").lower()
    
        ecol_ap = "merr_ap_{}".format(basecol)
        flagcol_ap = "flag_{}_ap".format(basecol)
    
        mask_ap  = np.where(master_catalogue[ecol_ap]  == irac_mag)[0]
    
        print('  Aperture magnitude')
        master_catalogue = flag_mag(master_catalogue, flagcol_ap, mask_ap)
    return master_catalogue

## 2. Outliers

In [7]:
def outliers_loopable(master_catalogue):
    for band_of_a_kind in all_bands:
        for band1, band2 in itertools.combinations(band_of_a_kind, 2):
            #print(band1, band2)
    
            basecol1, basecol2 = band1.replace(" ", "_").lower(), band2.replace(" ", "_").lower()
            
            # Aperture mag
            col1, col2 = "m_ap_{}".format(basecol1), "m_ap_{}".format(basecol2)
            ecol1, ecol2 = "merr_ap_{}".format(basecol1), "merr_ap_{}".format(basecol2)
            flagcol1, flagcol2 = "flag_ap_{}".format(basecol1), "flag_ap_{}".format(basecol2)
        
            try:
                master_catalogue = flag_outliers(master_catalogue, col1, col2,
                          ecol1, ecol2,
                          flagcol1, flagcol2,
                          labels=("{} (aperture)".format(band1), "{} (aperture)".format(band2)))
            except KeyError:
                print("One of {} and {} is not present in the catalogue.".format(col1, col2))
                      
        
            # Tot mag
            col1, col2 = "m_{}".format(basecol1), "m_{}".format(basecol2)              
            ecol1, ecol2 = "merr_{}".format(basecol1), "merr_{}".format(basecol2)              
            flagcol1, flagcol2 = "flag_{}".format(basecol1), "flag_{}".format(basecol2)
              
            try:
                master_catalogue = flag_outliers(master_catalogue, col1, col2, 
                          ecol1, ecol2,
                          flagcol1, flagcol2,
                          labels=("{} (total)".format(band1), "{} (total)".format(band2)))   
            except KeyError:
                print("One of {} and {} is not present in the catalogue.".format(col1, col2))  
    return master_catalogue
                      

## 3. Loop over table


In [8]:
for tile in file_list:
    master_catalogue = Table.read(tile)
    master_catalogue = panstars_loopable(master_catalogue)
    master_catalogue = irac_loopable(master_catalogue)
    master_catalogue = outliers_loopable(master_catalogue)
    file_num = tile.split('_')[6].split('.')[0]
    
    flag_cols = ["help_id"]
    for col in master_catalogue.colnames:
        if col.startswith("flag_"):
            flag_cols += [col]
    new_catname = "./data/tiles/{}_{}_{}_flags.fits".format(FIELD.lower(),SUFFIX, file_num)
    master_catalogue[flag_cols].write(new_catname, overwrite = True)

GPC1 g
  Aperture magnitude
    Number of flagged objects: 1
  Total magnitude
    Number of flagged objects: 1
GPC1 r
  Aperture magnitude
    Number of flagged objects: 4
  Total magnitude
    Number of flagged objects: 5
GPC1 i
  Aperture magnitude
    Number of flagged objects: 5
  Total magnitude
    Number of flagged objects: 8
GPC1 z
  Aperture magnitude
    Number of flagged objects: 3
  Total magnitude
    Number of flagged objects: 3
GPC1 y
  Aperture magnitude
    Number of flagged objects: 10
  Total magnitude
    Number of flagged objects: 9
IRAC i1
  Aperture magnitude
    Number of flagged objects: 0
IRAC i2
  Aperture magnitude
    Number of flagged objects: 0
IRAC i3
  Aperture magnitude
    Number of flagged objects: 0
IRAC i4
  Aperture magnitude
    Number of flagged objects: 0
DECam g (aperture) - Megacam g (aperture):
  Number of outliers: 14
DECam g (total) - Megacam g (total):
  Number of outliers: 17
SUPRIME g (aperture) - Megacam g (aperture):
  Number of outl